# Juliaで強化学習Tutorial

ここではJuliaのコーディングを学びながら強化学習の問題設定・アルゴリズムについて学んで行くためのTutorialを書き記す。

## 1. 強化学習の問題設定

強化学習とは、環境(与えられるタスクと報酬体系)とエージェント(タスクをこなすための戦略(Policy)を持っている存在)からなる系である。「(エージェントが最初に環境の情報を知らない状態からスタートし、試行錯誤を繰り返すうちに)ある環境の中で最終的に得られる報酬を最大化するような戦略を見つける」という問題を強化学習と呼ぶ。

ふんわりとした数式で一旦定義しておこう。

環境$\mathcal{E}$(まずは簡単のために決定論的マルコフ的な環境)は
 $$ \mathcal{E} = \{\mathcal{A} , \mathcal{S}, T(s,a) , g(a,s)\}, \ \ \ \ (s' = T(s,a), a\in\mathcal{A}, s,s'\in \mathcal{S} )$$
で書くことが出来る。$\mathcal{A}$はエージェントの取ることが出来る行動空間であり、$\mathcal{S}$は環境の取りうる状態空間, T(s,a)は状態sの時に行動aをとった時に次の遷移した状態である。$g(a,s)$は状態sの時にエージェントが行動aを起こした時にエージェントに与えられる報酬である。

一方で、エージェントとは(一般に確率的な行動指針)ポリシー$\pi$を持っていて、
 $$ p_a = \pi(s, a) \ \ \ \ \sum_a p_a = 1$$
と書ける。環境の状態がsの時に行動aを$p_a$の確率で取るという戦略になっている。エージェントも決定的な行動をとる場合は、$p_a$はone-hotベクトル(ある一つの要素が1でそれ以外が0)

また一般にはポリシー$\pi$を決める為の行動価値関数$Q(a,s)$を持っていて、例えば決定的な行動をとる時は$[p_a] = \mathrm{onehot(argmax_a(Q(a,s)))}$のように、行動価値関数からポリシーが決定される。強化学習の問題においては、エージェントの持っている行動価値関数Q(a,s)は最初出鱈目(例えば0の値で初期化されている)であり、エージェントが試行錯誤をしていく上でこの関数を更新していき、結果ポリシーを更新していくことが多い。

また、これをエージェントに含めるかは諸説あるが、エージェントの戦略のアップデートするためのアルゴリズムB(ベルマン作用素)も持たせてよくて、
$$ \pi' = B_p \pi$$
もしくは行動価値関数Q(a,s)を更新する 
$$ Q'(a,s) = B_Q Q(a,s)$$
と書け、例えば無限回アップデートを繰り返した後に辿り着く$ \pi^* = B^{\infty}\pi $が最善戦略(獲得報酬が最大化されている戦略)になっているようにBを設定する事が当然望ましい。



ここで一旦簡単な例を考えよう。１ターンに一度0~9の数字を選び、紙に書く。初期状態で既に0が書いてあり、２ターン与えられ、連続して数字を書き、その２桁の数字を報酬として２ターン目に与えられるというゲームを考える。

この時、$\mathcal{A} = \{0,1,\dots,9\} (|\mathcal{A}|=10)$、$\mathcal{S} = \{0,00,\dots,99\}, |\mathcal{S}|=111$であり、$T(s_{t-1}=n,a=m)=s'(=10^{t}m+n)$、$g(s_{t-1}=n,a=m) = \delta_{t,2} (10^{t}m+n)$が報酬として与えられる。

また最善の戦略は状況に関わらず、9を選択することなので、
$$[p^*_a] = [P(s,a)] = [P[a]] = [0,\dots, 0, 1]$$
である。

なんとなくイメージが付いてきただろうか？今回は簡単のためポリシーをアプデートするためのアルゴリズムについては考えなかった。次のセクションでは、一番原初的なアルゴリズムとして、ε-greedy法について学んでいく。

## 2. バンディット問題とε-greedy法
